In [5]:
pip install langchain

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.7 MB 8.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.6/18.7 MB 3.5 MB/s eta 0:00:05
   ---- ----------------------------------- 2.1/18.7 MB 3.7 MB/s eta 0:00:05
   ------ --------------------------------- 2.9/18.7 MB 3.7 MB/s eta 0:00:05
   -------- ------------------------------- 3.9/18.7 MB 3.8 MB/s eta 0:00:04
   ---------- ----------------------------- 4.7/18.7 MB 3.8 MB/s eta 0:00:04
   ----------- ---------------------------- 5.5/18.7 MB 3.8 MB/s eta 0:00:04
   ------------ --------------------------- 6.0/18.7 MB 3.8 MB/s eta 0:00:04
   --------------- ------------------------ 7.1/18.7 MB 3.8 MB/s eta 0:00:04
   ---------------- ----------------------- 7.9/18.7 MB 3.8 MB/s eta 0:00:03
   ------------------ --------------------- 8.7/18.7 MB 3.8 MB/s eta 0:00:03
   -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
import fitz  # PyMuPDF
import re
import unicodedata
import pytesseract
from PIL import Image

In [3]:
import fitz

In [ ]:
for page in doc:
    text= page.get_text()
    print(text)

In [ ]:
doc1 = fitz.open("gatwick-park-discharge-brochure.pdf")

In [ ]:
for page in doc1:
    text= page.get_text()
    print(text)

In [ ]:
doc3 = fitz.open("Home Care After Surgery.pdf")

In [ ]:
for page in doc3:
    text = page.get_text()
    print(text)

In [ ]:
doc4=fitz.open("patient_discharge_1540841480-3349_26074.pdf")

In [ ]:
for page in doc4:
    text = page.get_text()
    print(text)

In [ ]:
doc5 = fitz.open("Pharmacy First Advanced Service common conditions.pdf")

In [ ]:
for page in doc5:
    text = page.get_text()
    print(text)

In [ ]:
doc6 = fitz.open("post-operative-care.pdf")

In [ ]:
for page in doc6:
    text = page.get_text()
    print(text)

In [ ]:
doc7=fitz.open("Surgery Packet 2022.pdf")

In [ ]:
for page in doc7:
    text = page.get_text()
    print(text)

In [ ]:
for page in doc7:
    text = page.get_text()
    cleaned_text = clean_text(text)
    print(cleaned_text)

In [ ]:
doc8= fitz.open("WHO-HIS-SDS-2015.18-eng.pdf")

In [ ]:
for page in doc8:
    text = page.get_text()
    cleaned_text = clean_text(text)
    print(cleaned_text)

In [1]:
def clean_text(text: str) -> str:
    # Remove replacement char (� = U+FFFD)
    text = text.replace("\uFFFD", "")
    text=text.replace("", "")
    text = text.replace("*","")


    # Normalize ligatures, accents (e.g., ﬁ → fi)
    text = unicodedata.normalize("NFKC", text)

    # Remove multiple spaces/newlines
    text = re.sub(r"\s+", " ", text)

    return text.strip()

In [3]:
def clean_text(text: str) -> str:
    text = text.replace("\uFFFD", "")
    text=text.replace("", "")
    text = text.replace("*","")
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [5]:
def ocr_page(page) -> str:
    """Run OCR on a page using Tesseract."""
    pix = page.get_pixmap(dpi=300)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    return pytesseract.image_to_string(img)

In [7]:
def extract_text_from_pdf(pdf_path):
    """Extracts Text From Pdf using PymuPdf"""
    doc =fitz.open(pdf_path)
    all_text = []
    for page in doc:
        text = page.get_text()
        text = clean_text(text)

        if not text:
            text = ocr_page(page)
        all_text.append(text)
        
    return "\n".join(all_text)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text, chunk_size=500, overlap=50):
    """
    Split text into chunks using LangChain's RecursiveCharacterTextSplitter.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = splitter.split_text(text)
    return chunks

In [11]:
def process_folder(input_folder: str, output_folder: str, chunk_size=500, overlap=50):
    """Process all PDFs: extract, clean, save text + chunked JSON."""
    os.makedirs(output_folder, exist_ok=True)
    chunks_data = []  # collect all chunks for JSON

    for file in os.listdir(input_folder):
        if file.lower().endswith(".pdf"):
            pdf_path = os.path.join(input_folder, file)
            print(f"📄 Processing: {file}")

            # Step 1: Extract + clean text
            text = extract_text_from_pdf(pdf_path)

            # Step 2: Save raw text
            raw_out = os.path.splitext(file)[0] + ".txt"
            with open(os.path.join(output_folder, raw_out), "w", encoding="utf-8") as f:
                f.write(text)

            # Step 3: Chunk text
            chunks = chunk_text(text, chunk_size, overlap)

            # Collect chunks with metadata
            for i, chunk in enumerate(chunks):
                chunks_data.append({
                    "source": file,
                    "chunk_id": f"{file}_page_{i}",
                    "page_char_count":len(chunk),
                    "page_word_count": len(chunk.split(" ")),
                    "content": chunk
                })

            print(f"✅ Saved {len(chunks)} chunks for {file}")

    # Step 4: Save all chunks to JSON
    chunks_file = os.path.join(output_folder, "chunks.json")
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(chunks_data, f, ensure_ascii=False, indent=2)

    print(f"\n🎉 All PDFs processed! Chunks saved to {chunks_file}")

In [13]:
import os

In [19]:
process_folder("data","chunk")

📄 Processing: A Guide to Post Operative Care.pdf
✅ Saved 41 chunks for A Guide to Post Operative Care.pdf
📄 Processing: appendectomy.pdf
✅ Saved 7 chunks for appendectomy.pdf
📄 Processing: Appendicitis.pdf
✅ Saved 11 chunks for Appendicitis.pdf
📄 Processing: BIDMC_Post_Surgery_Guidelines.pdf
✅ Saved 7 chunks for BIDMC_Post_Surgery_Guidelines.pdf
📄 Processing: Caesarean section.pdf
✅ Saved 21 chunks for Caesarean section.pdf
📄 Processing: cholecystectomy.pdf
✅ Saved 17 chunks for cholecystectomy.pdf
📄 Processing: coronary artery bypass graft.pdf
✅ Saved 22 chunks for coronary artery bypass graft.pdf
📄 Processing: Diabetes and Your Lifestyle.pdf
✅ Saved 16 chunks for Diabetes and Your Lifestyle.pdf
📄 Processing: diabetes.pdf
✅ Saved 16 chunks for diabetes.pdf
📄 Processing: gatwick-park-discharge-brochure.pdf
✅ Saved 45 chunks for gatwick-park-discharge-brochure.pdf
📄 Processing: Hernia repair.pdf
✅ Saved 27 chunks for Hernia repair.pdf
📄 Processing: High blood pressure.pdf
✅ Saved 9 chun

In [21]:
import pandas as pd
dic = pd.read_json("chunk/chunks.json")

In [23]:
dic.head()

,source,chunk_id,page_char_count,page_word_count,content
0,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_0,499,64,A Guide to Post Operative Care - Post Operativ...
1,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_1,370,57,Headaches Headaches and Migraines Secondary He...
2,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_2,400,68,. Post-operative care begins once you arrive i...
3,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_3,455,71,". However, at some point this responsibility s..."
4,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_4,415,72,. Regular checks on the surgical site will als...


In [25]:
df = pd.DataFrame(dic)

In [27]:
df.head()

,source,chunk_id,page_char_count,page_word_count,content
0,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_0,499,64,A Guide to Post Operative Care - Post Operativ...
1,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_1,370,57,Headaches Headaches and Migraines Secondary He...
2,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_2,400,68,. Post-operative care begins once you arrive i...
3,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_3,455,71,". However, at some point this responsibility s..."
4,A Guide to Post Operative Care.pdf,A Guide to Post Operative Care.pdf_page_4,415,72,. Regular checks on the surgical site will als...


In [32]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [29]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

C:\Users\udaya\anaconda3\envs\py310env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[ 6.76569641e-02  6.34958968e-02  4.87131327e-02  7.93049932e-02
   3.74479853e-02  2.65277526e-03  3.93749401e-02 -7.09840422e-03
   5.93614280e-02  3.15369591e-02  6.00980893e-02 -5.29051833e-02
   4.06068265e-02 -2.59308796e-02  2.98428163e-02  1.12684851e-03
   7.35149309e-02 -5.03819138e-02 -1.22386634e-01  2.37028524e-02
   2.97265034e-02  4.24768254e-02  2.56337989e-02  1.99519284e-03
  -5.69191016e-02 -2.71598455e-02 -3.29035446e-02  6.60248697e-02
   1.19007058e-01 -4.58790585e-02 -7.26214871e-02 -3.25839929e-02
   5.23413904e-02  4.50552516e-02  8.25305562e-03  3.67023498e-02
  -1.39415562e-02  6.53919354e-02 -2.64272839e-02  2.06388984e-04
  -1.36643872e-02 -3.62809822e-02 -1.95043311e-02 -2.89738625e-02
   3.94270457e-02 -8.84090886e-02  2.62431079e-03  1.36714112e-02
   4.83063310e-02 -3.11565101e-02 -1.17329173e-01 -5.11690043e-02
  -8.85288417e-02 -2.18962971e-02  1.42986458e-02  4.44168039e-02
  -1.34814847e-02  7.43392557e-02  2.66382769e-02 -1.98762212e-02
   1.79190

In [31]:
import torch

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [37]:
embedding_model = SentenceTransformer(model_name_or_path='all-MiniLM-L6-v2',device=device)

In [41]:
import tqdm

In [49]:
text_dic = dict(dic)

In [59]:
text_dic

{'source': 0      A Guide to Post Operative Care.pdf
 1      A Guide to Post Operative Care.pdf
 2      A Guide to Post Operative Care.pdf
 3      A Guide to Post Operative Care.pdf
 4      A Guide to Post Operative Care.pdf
                       ...                
 601           WHO-HIS-SDS-2015.18-eng.pdf
 602           WHO-HIS-SDS-2015.18-eng.pdf
 603           WHO-HIS-SDS-2015.18-eng.pdf
 604           WHO-HIS-SDS-2015.18-eng.pdf
 605           WHO-HIS-SDS-2015.18-eng.pdf
 Name: source, Length: 606, dtype: object,
 'chunk_id': 0      A Guide to Post Operative Care.pdf_page_0
 1      A Guide to Post Operative Care.pdf_page_1
 2      A Guide to Post Operative Care.pdf_page_2
 3      A Guide to Post Operative Care.pdf_page_3
 4      A Guide to Post Operative Care.pdf_page_4
                          ...                    
 601           WHO-HIS-SDS-2015.18-eng.pdf_page_1
 602           WHO-HIS-SDS-2015.18-eng.pdf_page_2
 603           WHO-HIS-SDS-2015.18-eng.pdf_page_3
 604        

In [67]:
import json
from sentence_transformers import SentenceTransformer

# Load your JSON file
with open('chunk/chunks.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Load the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for each content item
for item in data:
    if "content" in item:
        item["embedding"] = model.encode(item["content"], convert_to_numpy=True).tolist()

In [ ]:
data["embedding"]